In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#@title
import time
import numpy as np
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.externals import joblib
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, CuDNNLSTM, Bidirectional
from keras.layers.embeddings import Embedding
## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
# Others
import nltk
from nltk.corpus import stopwords
from sklearn.manifold import TSNE
from keras.optimizers import SGD,Adam
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

In [0]:
#@title
train_folder = "./drive/My Drive/datasets/train-articles" # check that the path to the datasets folder is correct, 
dev_folder = "./drive/My Drive/datasets/dev-articles"     # if not adjust these variables accordingly
train_labels_file = "./drive/My Drive/datasets/train-task2-TC.labels"
dev_template_labels_file = "./drive/My Drive/datasets/dev-task-TC-template.out"
task_TC_output_file = "TFIDF_LR.txt"

def read_predictions_from_file(filename):
    """
    Reader for the gold file and the template output file. 
    Return values are four arrays with article ids, labels 
    (or ? in the case of a template file), begin of a fragment, 
    end of a fragment. 
    """
    articles_id, span_starts, span_ends, gold_labels = ([], [], [], [])
    with open(filename, "r") as f:
        for row in f.readlines():
            article_id, gold_label, span_start, span_end = row.rstrip().split("\t")
            articles_id.append(article_id)
            gold_labels.append(gold_label)
            span_starts.append(span_start)
            span_ends.append(span_end)
    return articles_id, span_starts, span_ends, gold_labels

In [0]:
#@title
# loading gold labels, articles ids and sentence ids from files *.task-TC.labels in the train labels folder 
ref_articles_id, ref_span_starts, ref_span_ends, train_gold_labels = read_predictions_from_file(train_labels_file)
print("Loaded %d annotations from %d articles" % (len(ref_span_starts), len(set(ref_articles_id))))

# reading data from the development set
dev_article_ids, dev_span_starts, dev_span_ends, dev_labels = read_predictions_from_file(dev_template_labels_file)

In [0]:
#@title
train=pd.read_csv('./drive/My Drive/datasets/train.csv').drop(['Unnamed: 0'],axis=1)
dev=pd.read_csv('./drive/My Drive/datasets/dev.csv').drop(['Unnamed: 0'],axis=1)
dev['Sentence']=dev['text']
del dev['text']

In [0]:
#@title
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [9]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2020-02-29 02:58:02--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-02-29 02:58:02--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-02-29 02:58:02--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
embeddings_index = dict()
f = open('./glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [11]:
'''import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))
train['Sentence']=train['Sentence'].apply(lambda x : ' '.join([word for word in x.split(' ') if (word not in stop_words and len(word)>1)]))
dev['Sentence']=dev['Sentence'].apply(lambda x : ' '.join([word for word in x.split(' ') if (word not in stop_words and len(word)>1)]))'''

'import nltk\nnltk.download(\'stopwords\')\nstop_words = set(stopwords.words("english"))\ntrain[\'Sentence\']=train[\'Sentence\'].apply(lambda x : \' \'.join([word for word in x.split(\' \') if (word not in stop_words and len(word)>1)]))\ndev[\'Sentence\']=dev[\'Sentence\'].apply(lambda x : \' \'.join([word for word in x.split(\' \') if (word not in stop_words and len(word)>1)]))'

In [0]:
(sorted(train['Sentence'].str.split().apply(len)))[-10:]

[83, 87, 89, 92, 98, 101, 109, 109, 147, 157]

In [0]:
vocabulary_size = 8300
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(train['Sentence'].append(dev['Sentence']))
sequences = tokenizer.texts_to_sequences(train['Sentence'])
data = pad_sequences(sequences, maxlen=90)

In [0]:
OOV=[]
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1 :
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
        else:
            OOV.append(word)

In [0]:
ttfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,ngram_range=(1,6),stop_words='english',analyzer='char',max_features=2750)
ttfidf_w = TfidfVectorizer(sublinear_tf=True, min_df=1,ngram_range=(1,1),stop_words='english',analyzer='word',max_features=3000)
len_v= train['Span'].values.reshape(-1,1)
DF=pd.concat([train,dev])
features = ttfidf.fit_transform(DF.Sentence).toarray().astype('float64')
features_w = ttfidf_w.fit_transform(DF.Sentence).toarray().astype('float64')
features = np.append(features,DF['Span'].values.reshape(-1,1),axis=1)
features= np.append(features,features_w,axis=1)

In [0]:
Train_X, Train_Y = features[:len(train)], DF[:len(train)]['Target']
Test=features[len(train):]

In [0]:
from sklearn import preprocessing 
from keras.utils import np_utils
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder()
y=label_encoder.fit_transform(train.Target)
y=np_utils.to_categorical(y)
test_sequences = tokenizer.texts_to_sequences(dev['Sentence'])
test_data = pad_sequences(test_sequences, maxlen=90) 

In [0]:
def pred_file(pred,x=True):
  if x:
    pred=label_encoder.inverse_transform(pred.argmax(axis=1)).tolist()
  task_TC_output_file='LSTM.txt'
  ##### writing predictions to file
  with open(task_TC_output_file, "w") as fout:
      for article_id, prediction, span_start, span_end in zip(dev_article_ids, pred, dev_span_starts, dev_span_ends):
          fout.write("%s\t%s\t%s\t%s\n" % (article_id, prediction, span_start, span_end))
  print("Predictions written to file " + task_TC_output_file)
  from google.colab import files
  files.download('./LSTM.txt')

In [0]:
def plot_model(history,rng=[0,-1]):
  plt.plot(history.history['acc'][rng[0]:rng[1]])
  plt.plot(history.history['val_acc'][rng[0]:rng[1]])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Valid'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(history.history['loss'][rng[0]:rng[1]])
  plt.plot(history.history['val_loss'][rng[0]:rng[1]])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Valid'], loc='upper left')
  plt.show()

def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [0]:
from keras.layers import Input, Embedding, LSTM, Dense, concatenate
from keras.models import Model
from keras.models import model_from_json
SAVED=True
#Functional Model
MAX_SEQUENCE_LENGTH=90
EMBEDDING_DIM=100

embedding_layer = Embedding(vocabulary_size,EMBEDDING_DIM,weights=[embedding_matrix],input_length=MAX_SEQUENCE_LENGTH,trainable=False)
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
embedded_sequences = Dropout(0.4)(embedded_sequences)
LSTM = CuDNNLSTM(100, return_sequences=True)(embedded_sequences)
LSTM = Dropout(0.4)(LSTM)
LSTM = CuDNNLSTM(100)(LSTM)
Output= Dense(14, activation='sigmoid')(LSTM)
opt = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, amsgrad=True, clipnorm=1)
model = Model(sequence_input,Output)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['acc'])
if SAVED:
  model.load_weights("./drive/My Drive/datasets/model.h5")
else:
  hist=model.fit(data, y, validation_split=0.2, epochs=100, batch_size=64, verbose=1)
  plot_model(hist)

In [0]:
pred=model.predict(data)
assemble=lambda features : np.concatenate(features,axis=1)
model_lr=LogisticRegression(penalty='l2', class_weight='balanced', solver="liblinear", max_iter=500)
model_lr.fit(Train_X, Train_Y)
pred_lr=model_lr.predict_proba(Train_X)
t_x=assemble([pred_lr,pred])

In [0]:
if not SAVED:
  # serialize model to JSON
  model_json = model.to_json()
  with open("model.json", "w") as json_file:
      json_file.write(model_json)
  # serialize weights to HDF5
  model.save_weights("model.h5")
  print("Saved model to disk")

In [0]:
meta = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,colsample_bynode=1, colsample_bytree=0.8, gamma=4,learning_rate=0.02, max_delta_step=0, max_depth=4,min_child_weight=5, n_estimators=600,njobs=4, objective='multi:softmax', num_classes=14, scale_pos_weight=1, seed=1234, subsample=0.7, verbosity=1)

In [39]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)


 All results:
{'mean_fit_time': array([197.52205485, 145.19374537, 145.14607829, 180.79496837,
       181.86118245, 200.24097097, 195.77097422, 204.44364834,
       133.00103068, 187.23906386, 180.53325224, 195.08657426]), 'std_fit_time': array([ 3.00392534,  8.60132374,  0.25341506,  1.12894079,  0.66717236,
        0.93027562,  0.2181088 ,  0.95091138,  0.21066781,  0.66887603,
        0.5650523 , 24.23537994]), 'mean_score_time': array([2.51676869, 2.35804206, 2.4240573 , 2.56846321, 2.30962676,
       2.42475122, 2.63364005, 2.3195281 , 2.19422662, 2.35056794,
       2.40110558, 1.13423133]), 'std_score_time': array([0.03030434, 0.20628705, 0.06444117, 0.08841918, 0.02714269,
       0.01374618, 0.03781214, 0.02613819, 0.04090508, 0.0542496 ,
       0.1741034 , 0.47413426]), 'param_subsample': masked_array(data=[0.8, 0.8, 0.8, 0.7, 0.7, 0.8, 0.7, 0.7, 0.7, 0.7, 0.8,
                   0.7],
             mask=[False, False, False, False, False, False, False, False,
                 

In [0]:
pred_lr=model_lr.predict_proba(Test)
preds=model.predict(test_data)

In [0]:
meta.fit(t_x,Train_Y)
meta_pred=meta.predict(assemble([pred_lr,preds]))

In [44]:
pred_file(meta_pred,x=False)

Predictions written to file LSTM.txt
